<a href="https://www.kaggle.com/code/siddharth050/encoder-decoder-summarization?scriptVersionId=172061824" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install textacy
!pip install contractions

In [ ]:
import textacy
from textacy import preprocessing as prep
import re
import contractions

In [ ]:
data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')

In [ ]:
data.columns

In [ ]:
data = data.drop(['id'],axis = 1)

In [ ]:
data.head()

In [ ]:
data.iloc[0,0]

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
sen = data.sample().iloc[0,1]

In [ ]:
sen

In [ ]:
# defining regex function for the pipeline

#removing \n
def remove_n(text):
    return re.sub(r"\n"," ",text)

#removing html tags
def remove_html(text):
    text = re.sub(r"<[^>]+>"," ",text)
    return re.sub(r"&[\S]+"," ",text) #removes &nbsp
#removing emails
def remove_email(text):
    return re.sub(r"\S*@\S*.com","",text)

#removing hashtags
def remove_hash(text):
    return re.sub(r"\#*","",text)

In [ ]:
#Expand Contracions
def expand(text):
    expanded = []
    for word in text.split():
        expanded.append(contractions.fix(word))
    txt = ' '.join(expanded)
    return txt

In [ ]:
#Lowering word
def low(text):
    return text.lower()

In [ ]:
pipe = prep.make_pipeline(
    remove_email,
    prep.replace.emojis,
    prep.replace.urls,
    prep.replace.phone_numbers,
    prep.replace.hashtags,
    prep.replace.currency_symbols,
    remove_html,
    prep.remove.brackets,
    low,
    remove_n,
    prep.normalize.hyphenated_words, # sentences that have been split by a '-' is split and joined together
    prep.normalize.quotation_marks, # normalize all singal and double quotes in the text to ASCII representation,
    prep.normalize.unicode,
    prep.normalize.bullet_points,
    prep.normalize.whitespace
)

In [ ]:
def pre_process(df):
    df['article'] = df['article'].apply(pipe)
    df['article'] = df['article'].apply(expand)

    df['highlights'] = df['highlights'].apply(pipe)
    df['highlights'] = df['highlights'].apply(expand)
    return df

In [ ]:
data.sample(frac=0.35).reset_index(drop=True).shape

In [ ]:
txt = data.sample(frac=0.01).reset_index(drop=True) # sample data to show output of preprocess pipeline

In [ ]:
df = data.sample(frac=0.35).reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
txt.iloc[0,0]

In [ ]:
txt = pre_process(txt)

In [ ]:
txt.iloc[0,0]

In [ ]:
df = pre_process(df)

In [ ]:
df.to_csv('output.csv')

In [ ]:
df = pd.read_csv('/kaggle/input/cnn-pos-tagged/output.csv')

## Mandate 3

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
sen = txt.sample().iloc[0,0]

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
sen

In [ ]:
tokens = word_tokenize(sen)

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
len(stop_words)

In [ ]:
league_leveltagged = nltk.pos_tag(tokens)

In [ ]:
s = ""
for i in tagged:
    s += i[1]
    s+=' '

In [ ]:
s

In [ ]:
len(s),len(sen)

In [ ]:
def tagging(text):
    tokens = word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    s = ""
    for i in tagged:
        s += i[1]
        s+=' '
    return s

In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text)
    s = ""
    for i in tokens:
        if i.casefold() not in stop_words:
            s += i
            s+=' '
    return s

In [ ]:
def remove_special(text):
    return re.sub(r'[^a-zA-Z\s]',' ',text)    

In [ ]:
sen = txt.sample().iloc[0,0]

In [ ]:
sen

In [ ]:
sen = prep.normalize.whitespace(remove_stop_words(remove_special(sen)))

In [ ]:
sen

In [ ]:
tagging(sen)

In [ ]:
df['article'] = df['article'].apply(remove_special)

In [ ]:
df['article'] = df['article'].apply(remove_stop_words)

In [ ]:
df['article'] = df['article'].apply(prep.normalize.whitespace)

In [ ]:
df['tagged_article'] = df['article'].apply(tagging)

In [ ]:
df['highlights'] = df['highlights'].apply(remove_special)

In [ ]:
df['highlights'] = df['highlights'].apply(remove_stop_words)

In [ ]:
df['highlights'] = df['highlights'].apply(prep.normalize.whitespace)

In [ ]:
df.sample()

In [ ]:
df.to_csv('/kaggle/working/special_and_stop_removed_pos.csv',index=False)

In [ ]:
data = pd.read_csv('/kaggle/input/cnn-pos-tagged/special_and_stop_removed_pos.csv')

In [ ]:
df = data.sample(frac=0.5).reset_index(drop=True)

In [ ]:
df.to_csv('/kaggle/working/df-50k.csv')

In [ ]:
df = pd.read_csv('/kaggle/input/cnn-pos-tagged/df-50k.csv')

In [ ]:
df = df.drop('Unnamed: 0',axis=1)

In [ ]:
a,s,p = df.sample().iloc[0,:]

In [ ]:
a

In [ ]:
s

In [ ]:
p

In [ ]:
df.sample().iloc[0,2]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
text_tokenizer = Tokenizer(oov_token='<OOV>')
pos_tokenizer = Tokenizer(oov_token='<OOV>')

In [ ]:
ar , p = df.iloc[0,[0,2]]

In [ ]:
text_tokenizer.fit_on_texts([ar])

In [ ]:
text_tokenizer.word_index

In [ ]:
ar

In [ ]:
len(text_tokenizer.texts_to_sequences([ar])[0])

In [ ]:
pad_tokenizer.fit_on_texts([p])

In [ ]:
p

In [ ]:
pad_tokenizer.word_index

In [ ]:
pad_tokenizer.texts_to_sequences([p])

In [ ]:
df.head()

## Word2Vec

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
df.columns

In [ ]:
# corp = pd.concat([df['article'],df['highlights']],axis=0).reset_index(drop=True)
corp_article = df['article']
corp_summary = df['highlights']
corp_pos = df['tagged_article']

In [ ]:
# token_corp = corp.apply(word_tokenize)
token_corp_article = corp_article.apply(word_tokenize)
token_corp_summary = corp_summary.apply(word_tokenize)
token_corp_pos = corp_pos.apply(word_tokenize)

In [ ]:
# token_corp_list = token_corp.tolist()
token_corp_article_list = token_corp_article.tolist()
token_corp_summary_list = token_corp_summary.tolist()
token_corp_pos_list = token_corp_pos.tolist()

In [ ]:
token_corp_summary_list[0]

In [ ]:
model_article = Word2Vec(sentences=token_corp_article_list,vector_size=300,window=10,min_count=1,sg=1)

In [ ]:
model_article.save('word2vec_50k_article.model')

In [ ]:
model_summary = Word2Vec(sentences=token_corp_summary_list,vector_size=300,window=10,min_count=1,sg=1)

In [ ]:
model_summary.save('word2vec_50k_summary.model')

In [ ]:
model_pos = Word2Vec(sentences=token_corp_pos_list,vector_size=100,window=10,min_count=1,sg=1)

In [ ]:
model_pos.save('word2vec_50k_pos.model')

In [ ]:
# model = Word2Vec(sentences=token_corp_list,vector_size=100,window=10,min_count=1,sg=1)

In [ ]:
model.save('word2vec_skipgram.model')

In [ ]:
model = Word2Vec.load('/kaggle/input/word2vec-text-summarization-model/word2vec_skipgram.model')

In [ ]:
model.wv['tax']

In [ ]:
model.wv['criminal']

In [ ]:
model_a = Word2Vec.load('/kaggle/input/word2vec-text-summarization-model/word2vec_50k_article.model')

In [ ]:
model_s = Word2Vec.load('/kaggle/input/word2vec-text-summarization-model/word2vec_50k_summary.model')

In [ ]:
model_p = Word2Vec.load('/kaggle/input/word2vec-text-summarization-model/word2vec_50k_pos.model')

In [ ]:
model_a.wv['tax']

In [ ]:
model_s.wv['tax']

In [ ]:
model_p.wv['NN']

In [ ]:
word_index = {word : index for index,word in enumerate(model.wv.index_to_key)}

In [ ]:
word_vector = model.wv
vocab_size = len(word_vector)
vector_size = word_vector.vector_size
vocab_size,vector_size

In [ ]:
embedding_matrix = np.zeros((vocab_size, vector_size))
for i, word in enumerate(word_vector.key_to_index.keys()):
    embedding_vector = word_vector[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
model.wv['one']

In [ ]:
embedding_matrix[1]

In [ ]:
start_vector = np.random.rand(vector_size)
end_vector = np.random.rand(vector_size)
embedding_matrix_extended = np.vstack([embedding_matrix, start_vector, end_vector])
vocab_size_extended = embedding_matrix_extended.shape[0]

In [ ]:
word_to_index_extended = {word: idx for idx, word in enumerate(word_vector.key_to_index.keys())}
word_to_index_extended['<start>'] = vocab_size
word_to_index_extended['<end>'] = vocab_size+1

In [ ]:
word_to_index_extended['<start>']

In [ ]:
def txt_to_seq(text,word_index):
    tokens = text.split()
    seq = [word_index[i] for i in tokens if i in word_index]
    return seq

In [ ]:
word2vec_text = []
for i in df['article']:
    word2vec_text.append(txt_to_seq(i,word_to_index_extended))

In [ ]:
word2vec_summary = []
for i in df['highlights']:
    word2vec_summary.append(txt_to_seq(i,word_to_index_extended))

In [ ]:
len(word2vec_text[0]),len(word2vec_summary[0])

### seperate models

In [ ]:
word_index_article = {word : index for index,word in enumerate(model_a.wv.index_to_key)}
word_index_summary = {word : index for index,word in enumerate(model_s.wv.index_to_key)}
word_index_pos = {word : index for index,word in enumerate(model_p.wv.index_to_key)}

In [ ]:
word_vector_article = model_a.wv
word_vector_summary = model_s.wv
word_vector_pos = model_p.wv

vocab_size_article = len(word_vector_article)
vocab_size_summary = len(word_vector_summary)
vocab_size_pos = len(word_vector_pos)

vector_size_article = word_vector_article.vector_size
vector_size_summary = word_vector_summary.vector_size
vector_size_pos = word_vector_pos.vector_size

vocab_size_article,vector_size_article,vocab_size_summary,vector_size_summary,vocab_size_pos,vector_size_pos,

In [ ]:
embedding_matrix_article = np.zeros((vocab_size_article, vector_size_article))
for i, word in enumerate(word_vector_article.key_to_index.keys()):
    embedding_vector = word_vector_article[word]
    if embedding_vector is not None:
        embedding_matrix_article[i] = embedding_vector

In [ ]:
embedding_matrix_summary = np.zeros((vocab_size_summary, vector_size_summary))
for i, word in enumerate(word_vector_summary.key_to_index.keys()):
    embedding_vector = word_vector_summary[word]
    if embedding_vector is not None:
        embedding_matrix_summary[i] = embedding_vector

In [ ]:
embedding_matrix_pos = np.zeros((vocab_size_pos, vector_size_pos))
for i, word in enumerate(word_vector_pos.key_to_index.keys()):
    embedding_vector = word_vector_pos[word]
    if embedding_vector is not None:
        embedding_matrix_pos[i] = embedding_vector

In [ ]:
start_vector_art = np.random.rand(vector_size_article)
end_vector_art = np.random.rand(vector_size_article)
embedding_matrix_article_extended = np.vstack([embedding_matrix_article, start_vector_art, end_vector_art])
vocab_article_size_extended = embedding_matrix_article_extended.shape[0]

In [ ]:
embedding_matrix_summary_extended = np.vstack([embedding_matrix_summary, start_vector_art, end_vector_art])
vocab_summary_size_extended = embedding_matrix_summary_extended.shape[0]

In [ ]:
start_vector_pos = np.random.rand(vector_size_pos)
end_vector_pos = np.random.rand(vector_size_pos)
embedding_matrix_pos_extended = np.vstack([embedding_matrix_pos, start_vector_pos, end_vector_pos])
vocab_pos_size_extended = embedding_matrix_pos_extended.shape[0]

In [ ]:
word_to_index_article_extended = {word: idx for idx, word in enumerate(word_vector_article.key_to_index.keys())}
word_to_index_article_extended['<start>'] = vocab_size_article
word_to_index_article_extended['<end>'] = vocab_size_article+1

In [ ]:
word_to_index_summary_extended = {word: idx for idx, word in enumerate(word_vector_summary.key_to_index.keys())}
word_to_index_summary_extended['<start>'] = vocab_size_summary
word_to_index_summary_extended['<end>'] = vocab_size_summary+1

In [ ]:
word_to_index_pos_extended = {word: idx for idx, word in enumerate(word_vector_pos.key_to_index.keys())}
word_to_index_pos_extended['<start>'] = vocab_size_pos
word_to_index_pos_extended['<end>'] = vocab_size_pos+1

In [ ]:
def txt_to_seq(text,word_index):
    tokens = text.split()
    seq = [word_index[i] for i in tokens if i in word_index]
    return seq

In [ ]:
word2vec_text = []
for i in df['article']:
    word2vec_text.append(txt_to_seq(i,word_to_index_article_extended))

In [ ]:
word2vec_summary = []
for i in df['highlights']:
    word2vec_summary.append(txt_to_seq(i,word_to_index_summary_extended))

In [ ]:
word2vec_pos = []
for i in df['tagged_article']:
    word2vec_pos.append(txt_to_seq(i,word_to_index_pos_extended))

In [ ]:
df.iloc[0,0]

In [ ]:
np.array(txt_to_seq(df.iloc[0,0],word_to_index_article_extended))

In [ ]:
len(df.iloc[0,0].split())

In [ ]:
len(word2vec_text[0])

In [ ]:
text_max = max([len(i) for i in word2vec_text])
summary_max = max([len(i) for i in word2vec_summary])
pad_max = max([len(i) for i in word2vec_pos])

In [ ]:
text_max,summary_max,pad_max

## Tokenizers for text and pos

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
text_tokenizer = Tokenizer(oov_token='<OOV>')

In [ ]:
pos_tokenizer = Tokenizer(oov_token='<OOV>') 

In [ ]:
text_tokenizer.fit_on_texts(df['article'] + df['highlights'])

In [ ]:
pos_tokenizer.fit_on_texts(df['tagged_article'])

In [ ]:
len(text_tokenizer.word_index)

In [ ]:
len(pos_tokenizer.word_index)

In [ ]:
m = max(text_tokenizer.word_index.values())
text_tokenizer.word_index['<start>'] = m+1
text_tokenizer.word_index['<end>'] = m+2

text_tokenizer.word_counts['<start>'] = 1
text_tokenizer.word_counts['<end>'] = 1

text_tokenizer.index_word[m+1] = '<start>'
text_tokenizer.index_word[m+2] = '<end>'

In [ ]:
m = max(pos_tokenizer.word_index.values())
pos_tokenizer.word_index['<start>'] = m+1
pos_tokenizer.word_index['<end>'] = m+2

pos_tokenizer.word_counts['<start>'] = 1
pos_tokenizer.word_counts['<end>'] = 1

pos_tokenizer.index_word[m+1] = '<start>'
pos_tokenizer.index_word[m+2] = '<end>'

In [ ]:
len(text_tokenizer.word_index)

In [ ]:
len(pos_tokenizer.word_index)

In [ ]:
tokenized_text = []
for i in df['article']:
    tokenized_text.append(text_tokenizer.texts_to_sequences([i])[0])

In [ ]:
tokenized_summary = []
for i in df['highlights']:
    tokenized_summary.append(text_tokenizer.texts_to_sequences([i])[0])

In [ ]:
tokenized_pos = []
for i in df['tagged_article']:
    tokenized_pos.append(pos_tokenizer.texts_to_sequences([i])[0])

In [ ]:
#tokenizer
text_vocab = len(text_tokenizer.word_index)+1
pos_vocab = len(pos_tokenizer.word_index)+1
text_vocab, pos_vocab

In [ ]:
#word2vec
pos_vocab = len(pos_tokenizer.word_index)+1
pos_vocab

In [ ]:
text_max = max([len(i) for i in tokenized_text])
summary_max = max([len(i) for i in tokenized_summary])
pad_max = max([len(i) for i in tokenized_pos])

In [ ]:
a,s,p=df.sample().iloc[0,:]

In [ ]:
a

In [ ]:
p

In [ ]:
s

In [ ]:
text_max, summary_max,pad_max

## Padding the tokens for word2vec

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#tokenizer
padded_text = pad_sequences(tokenized_text,maxlen=200,padding='post')
padded_summary = pad_sequences(tokenized_summary,maxlen=50,padding='post')
padded_pos = pad_sequences(tokenized_pos,maxlen=200,padding='post')

In [ ]:
#word2vec 
padded_text = pad_sequences(word2vec_text,maxlen=text_max,padding='post',value=word_to_index_article_extended['<end>'])
padded_summary = pad_sequences(word2vec_summary,maxlen=summary_max,padding='post',value=word_to_index_summary_extended['<end>'])
padded_pos = pad_sequences(word2vec_pos,maxlen=pad_max,padding='post',value=word_to_index_pos_extended['<end>'])

In [ ]:
padded_text.shape, padded_pos.shape ,padded_summary.shape

In [ ]:
#text tokenizer
s = text_tokenizer.word_index['<start>']
e = text_tokenizer.word_index['<end>']
s,e

In [ ]:
#word2vec
s_a = word_to_index_article_extended['<start>']
e_a = word_to_index_article_extended['<end>']

s_s = word_to_index_summary_extended['<start>']
e_s = word_to_index_summary_extended['<end>']

s_p = word_to_index_pos_extended['<start>']
e_p = word_to_index_pos_extended['<end>']
s_a,e_a , s_s,e_s, s_p,e_p

In [ ]:
padded_text[0]

In [ ]:
padded_summary[0]

In [ ]:
padded_pos[0]

In [ ]:
padded_text = np.insert(padded_text,[0,len(padded_text[0])],[s_a,e_a],axis=1)
padded_summary = np.insert(padded_summary,[0,len(padded_summary[0])],[s_s,e_s],axis=1)
padded_pos = np.insert(padded_pos,[0,len(padded_pos[0])],[s_p,e_p],axis=1)

In [ ]:
padded_text[0]

In [ ]:
padded_summary[0]

In [ ]:
padded_pos[0]

In [ ]:
# for tokenizer
p_s = pos_tokenizer.word_index['<start>']
p_e = pos_tokenizer.word_index['<end>']
p_s,p_e

In [ ]:
padded_pos[0]

In [ ]:
padded_pos = np.insert(padded_pos,[0,len(padded_pos[0])],[p_s,p_e],axis=1)

In [ ]:
padded_pos[0]

In [ ]:
padded_text.shape, padded_pos.shape ,padded_summary.shape

## Model Training

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU,LSTM, Embedding,TimeDistributed, Concatenate, Dense,Bidirectional,AdditiveAttention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam,RMSprop,SGD

### model for text tokenizer

In [ ]:
encoder_input = Input(shape= (None,))
enc_embedding = Embedding(text_vocab,100)(encoder_input)

pos_input = Input(shape=(None,))
pos_embedding = Embedding(pos_vocab,50)(pos_input)

encoder_input_combined = Concatenate()([enc_embedding,pos_embedding])

encoder_lstm_1 = Bidirectional(LSTM(128,return_sequences=True))(encoder_input_combined)
encoder_lstm_2 = LSTM(64,return_sequences=True)(encoder_lstm_1)
encoder_output, state_h, state_c = LSTM(64,return_state=True,return_sequences=True)(encoder_lstm_2)
print(encoder_output.shape)

decoder_input = Input(shape=(None,))
dec_emb_layer = Embedding(text_vocab, 100)
dec_embedding = dec_emb_layer(decoder_input)

decoder_lstm_1,_,_ = LSTM(128,return_sequences=True,return_state=True)(dec_embedding, initial_state = [state_h,state_c])
decoder_lstm_2,_,_ = LSTM(64,return_sequences=True,return_state=True)(decoder_lstm_1)
decoder_output,_,_ = LSTM(64,return_sequences=True,return_state=True)(decoder_lstm_2)
print(decoder_output.shape)
attention_layer = AdditiveAttention()
attention_out = attention_layer([decoder_output, encoder_output])
combined_output = Concatenate(axis=-1)([decoder_output, attention_out])

decoder_dense = TimeDistributed(Dense(text_vocab,activation='softmax'))
decoder_output = decoder_dense(combined_output)

model_txt = Model([encoder_input,pos_input,decoder_input],decoder_output)

### model for text and summary embeddings from word2vec

In [ ]:
encoder_input = Input(shape= (None,),name = 'encoder_input')
enc_emb_layer = Embedding(vocab_size_extended,100,trainable=False,name='enc_emb_layer')
enc_embedding = enc_emb_layer(encoder_input)

pos_input = Input(shape=(None,),name = 'pos_input')
pos_embedding = Embedding(pos_vocab,50, name='pos_embedding')(pos_input)

encoder_input_combined = Concatenate(name = 'encoder_input_combined')([enc_embedding,pos_embedding])

encoder_lstm_1 = LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_1')(encoder_input_combined)
encoder_lstm_2 = LSTM(32,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_2')(encoder_lstm_1)
encoder_lstm_3 = LSTM(32,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_3')(encoder_lstm_2)
encoder_output, state_h, state_c = LSTM(16,return_state=True,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_final')(encoder_lstm_3)

decoder_input = Input(shape=(None,),name='decoder_input')
dec_emb_layer = Embedding(vocab_size_extended, 100,trainable=False,name='dec_emb_layer')
dec_embedding = dec_emb_layer(decoder_input)

decoder_lstm_1,_,_ = LSTM(128,return_sequences=True,return_state=True,name='decoder_lstm_1')(dec_embedding, initial_state = [state_h,state_c])
decoder_lstm_2,_,_ = LSTM(32,return_sequences=True,return_state=True,name='decoder_lstm_2')(decoder_lstm_1)
decoder_lstm_3,_,_ = LSTM(32,return_sequences=True,return_state=True,name='decoder_lstm_3')(decoder_lstm_2)
decoder_output,_,_ = LSTM(16,return_sequences=True,return_state=True,name='decoder_lstm_final')(decoder_lstm_3)

attention_layer = AdditiveAttention(name='attention_layer')
attention_out = attention_layer([decoder_output, encoder_output])
combined_output = Concatenate(axis=-1,name='output_concat')([decoder_output, attention_out])

decoder_dense = TimeDistributed(Dense(vocab_size_extended,activation='softmax'))
decoder_output = decoder_dense(combined_output)

model_w2v = Model([encoder_input,pos_input,decoder_input],decoder_output)

enc_emb_layer.set_weights([embedding_matrix_extended])
dec_emb_layer.set_weights([embedding_matrix_extended])

In [ ]:
# plot_model(model)

In [ ]:
model_w2v.compile(optimizer=Adam(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model_w2v.summary()

### Model where all emeddings from word2vec

In [ ]:
encoder_input = Input(shape= (None,),name = 'encoder_input')
enc_emb_layer = Embedding(vocab_article_size_extended,100,trainable=False,name='enc_emb_layer')
enc_embedding = enc_emb_layer(encoder_input)

pos_input = Input(shape=(None,),name = 'pos_input')
pos_emb_layer = Embedding(vocab_pos_size_extended,50, name='pos_embedding')
pos_embedding = pos_emb_layer(pos_input)

encoder_input_combined = Concatenate(name = 'encoder_input_combined')([enc_embedding,pos_embedding])

encoder_lstm_1 = LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_1')(encoder_input_combined)
encoder_lstm_2 = LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_2')(encoder_lstm_1)
encoder_lstm_3 = LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_3')(encoder_lstm_2)
encoder_output, state_h, state_c = LSTM(32,return_state=True,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_final')(encoder_lstm_3)

decoder_input = Input(shape=(None,),name='decoder_input')
dec_emb_layer = Embedding(vocab_summary_size_extended, 100,trainable=False,name='dec_emb_layer')
dec_embedding = dec_emb_layer(decoder_input)

decoder_lstm_1,_,_ = LSTM(128,return_sequences=True,return_state=True,name='decoder_lstm_1')(dec_embedding, initial_state = [state_h,state_c])
decoder_lstm_2,_,_ = LSTM(64,return_sequences=True,return_state=True,name='decoder_lstm_2')(decoder_lstm_1)
decoder_lstm_3,_,_ = LSTM(64,return_sequences=True,return_state=True,name='decoder_lstm_3')(decoder_lstm_2)
decoder_output,_,_ = LSTM(32,return_sequences=True,return_state=True,name='decoder_lstm_final')(decoder_lstm_3)

attention_layer = AdditiveAttention(name='attention_layer')
attention_out = attention_layer([decoder_output, encoder_output])
combined_output = Concatenate(axis=-1,name='output_concat')([decoder_output, attention_out])

decoder_dense = TimeDistributed(Dense(vocab_summary_size_extended,activation='softmax'))
decoder_output = decoder_dense(combined_output)

model_all_w2v = Model([encoder_input,pos_input,decoder_input],decoder_output)

enc_emb_layer.set_weights([embedding_matrix_article_extended])
dec_emb_layer.set_weights([embedding_matrix_summary_extended])
pos_emb_layer.set_weights([embedding_matrix_pos_extended])

In [ ]:
model_all_w2v.compile(optimizer=Adam(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model_all_w2v.summary()

In [ ]:
encoder_input = Input(shape= (None,),name = 'encoder_input')
enc_emb_layer = Embedding(vocab_article_size_extended,100,trainable=False,name='enc_emb_layer')
enc_embedding = enc_emb_layer(encoder_input)

pos_input = Input(shape=(None,),name = 'pos_input')
pos_emb_layer = Embedding(vocab_pos_size_extended,50, name='pos_embedding')
pos_embedding = pos_emb_layer(pos_input)

encoder_input_combined = Concatenate(name = 'encoder_input_combined')([enc_embedding,pos_embedding])

encoder_lstm_1 = Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_1'))(encoder_input_combined)
encoder_lstm_2 = Bidirectional(LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_2'))(encoder_lstm_1)
encoder_lstm_3 = Bidirectional(LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_3'))(encoder_lstm_2)
encoder_output, state_h, state_c = LSTM(32,return_state=True,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_final')(encoder_lstm_3)

decoder_input = Input(shape=(None,),name='decoder_input')
dec_emb_layer = Embedding(vocab_summary_size_extended, 100,trainable=False,name='dec_emb_layer')
dec_embedding = dec_emb_layer(decoder_input)

decoder_lstm_1,_,_ = LSTM(128,return_sequences=True,return_state=True,name='decoder_lstm_1')(dec_embedding, initial_state = [state_h,state_c])
decoder_lstm_2,_,_ = LSTM(64,return_sequences=True,return_state=True,name='decoder_lstm_2')(decoder_lstm_1)
decoder_lstm_3,_,_ = LSTM(64,return_sequences=True,return_state=True,name='decoder_lstm_3')(decoder_lstm_2)
decoder_output,_,_ = LSTM(32,return_sequences=True,return_state=True,name='decoder_lstm_final')(decoder_lstm_3)

attention_layer = AdditiveAttention(name='attention_layer')
attention_out = attention_layer([decoder_output, encoder_output])
combined_output = Concatenate(axis=-1,name='output_concat')([decoder_output, attention_out])

decoder_dense = TimeDistributed(Dense(vocab_summary_size_extended,activation='softmax'))
decoder_output = decoder_dense(combined_output)

model_all_w2v_bi = Model([encoder_input,pos_input,decoder_input],decoder_output)

enc_emb_layer.set_weights([embedding_matrix_article_extended])
dec_emb_layer.set_weights([embedding_matrix_summary_extended])
pos_emb_layer.set_weights([embedding_matrix_pos_extended])

In [ ]:
model_all_w2v_bi.compile(optimizer=Adam(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model_all_w2v_bi.summary()

### 300 word2vec embedding with full sentence length final model

In [ ]:
encoder_input = Input(shape= (None,),name = 'encoder_input')
enc_emb_layer = Embedding(vocab_article_size_extended,300,trainable=False,name='enc_emb_layer')
enc_embedding = enc_emb_layer(encoder_input)

pos_input = Input(shape=(None,),name = 'pos_input')
pos_emb_layer = Embedding(vocab_pos_size_extended,100, trainable=False,name='pos_embedding')
pos_embedding = pos_emb_layer(pos_input)

encoder_input_combined_layer = Concatenate(name = 'encoder_input_combined')
encoder_input_combined = encoder_input_combined_layer([enc_embedding,pos_embedding])

encoder_lstm_1_layer = Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_1'),name='encoder_bi_1')
encoder_lstm_1 = encoder_lstm_1_layer(encoder_input_combined)
encoder_lstm_2_layer = LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_2')
encoder_lstm_2 = encoder_lstm_2_layer(encoder_lstm_1)
encoder_lstm_3_layer = LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_3')
encoder_lstm_3 = encoder_lstm_3_layer(encoder_lstm_2)
encoder_output_layer = LSTM(32,return_state=True,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,name='encoder_lstm_final')
encoder_output, state_h, state_c = encoder_output_layer(encoder_lstm_3)

decoder_input = Input(shape=(None,),name='decoder_input')
dec_emb_layer = Embedding(vocab_summary_size_extended, 300,trainable=False,name='dec_emb_layer')
dec_embedding = dec_emb_layer(decoder_input)

decoder_lstm_1_layer = LSTM(128,return_sequences=True,return_state=True,name='decoder_lstm_1')
decoder_lstm_1,_,_ = decoder_lstm_1_layer(dec_embedding, initial_state = [state_h,state_c])
decoder_lstm_2_layer = LSTM(64,return_sequences=True,return_state=True,name='decoder_lstm_2')
decoder_lstm_2,_,_ = decoder_lstm_2_layer(decoder_lstm_1)
decoder_lstm_3_layer = LSTM(64,return_sequences=True,return_state=True,name='decoder_lstm_3')
decoder_lstm_3,_,_ = decoder_lstm_3_layer(decoder_lstm_2)
decoder_output_layer = LSTM(32,return_sequences=True,return_state=True,name='decoder_lstm_final')
decoder_output,_,_ = decoder_output_layer(decoder_lstm_3)

attention_layer = AdditiveAttention(name='attention_layer')
attention_out = attention_layer([decoder_output, encoder_output])
combined_output_layer = Concatenate(axis=-1,name='output_concat')
combined_output = combined_output_layer([decoder_output, attention_out])

decoder_dense = TimeDistributed(Dense(vocab_summary_size_extended,activation='softmax',name='decoder_dense'),name='decoderTD')
decoder_output = decoder_dense(combined_output)

model_all_w2v_full_len = Model([encoder_input,pos_input,decoder_input],decoder_output)

enc_emb_layer.set_weights([embedding_matrix_article_extended])
dec_emb_layer.set_weights([embedding_matrix_summary_extended])
pos_emb_layer.set_weights([embedding_matrix_pos_extended])

In [ ]:
model_all_w2v_full_len.compile(optimizer=Adam(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model_all_w2v_full_len.summary()

In [ ]:
# train_text = padded_text[:10000,:]
# train_pos = padded_pos[:10000,:]
# train_summary = padded_summary[:10000,:]
train_text = padded_text[:,:]
train_pos = padded_pos[:,:]
train_summary = padded_summary[:,:]

In [ ]:
train_text.shape,train_pos.shape,train_summary.shape,

In [ ]:
decoder_input_data = train_summary[:,:-1]
decoder_target_data = train_summary[:,1:]
decoder_target_data = np.expand_dims(decoder_target_data,-1)

In [ ]:
#128 64 64 32 50k rows bidirectional 1st layer word2vec 300
history_bi = model_all_w2v_full_len.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=1,validation_split=0.2)

In [ ]:
# Save architecture
model_json = model_all_w2v_full_len.to_json()
with open('model_50k_full_len_architecture.json', 'w') as json_file:
    json_file.write(model_json)

# Save weights
model_all_w2v_full_len.save_weights('model_50k_full_len_weights.weights.h5')

In [ ]:
model_all_w2v_full_len.save('model_50k_full_len.h5')

In [ ]:
model_all_w2v_full_len.save_weights('model_50k_full_len_weights.weights.h5')

In [ ]:
model_json = model_all_w2v_full_len.to_json()
with open("model_50k_full_len.json", "w") as json_file:
    json_file.write(model_json)
model_all_w2v_full_len.save_weights("model_50k_full_len.weights.h5")

In [ ]:
#128 64 64 32 10k rows bidirectional
history_bi = model_all_w2v_bi.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=10,validation_split=0.2)

In [ ]:
model_all_w2v_bi.save('/kaggle/working/model_all_w2v_bi.keras')

In [ ]:
#128 64 64 32 10k rows
history = model_all_w2v.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=10,validation_split=0.2)

In [ ]:
model_all_w2v.save('/kaggle/working/model_all_w2v.keras')

In [ ]:
#128 64 64 32 10k rows
history = model_all_w2v.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=10,validation_split=0.2)

In [ ]:
#128 32 32 16 30k rows
history = model_all_w2v.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=10,validation_split=0.2)

In [ ]:
#256 128 128 16
# model.fit([padded_text,padded_pos,decoder_input_data],decoder_target_data,batch_size=32,epochs=25,validation_split=0.2)
history = model_w2v.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=5,validation_split=0.2)

In [ ]:
# 128 32 32 16
history = model_w2v.fit([train_text,train_pos,decoder_input_data],decoder_target_data,epochs=5,validation_split=0.2)

In [ ]:
enc_inp_1 = Input(shape= (None,))
enc_embedd = enc_emb_layer(enc_inp_1)

enc_inp_2= Input(shape=(None,))
pos_embedd = pos_emb_layer(enc_inp_2)

enc_inp_comb = encoder_input_combined_layer([enc_embedd,pos_embedd])
e_1 = encoder_lstm_1_layer(enc_inp_comb)
e_2 = encoder_lstm_2_layer(e_1)
e_3 = encoder_lstm_3_layer(e_2)
enc_out, h, c = encoder_output_layer(e_3)

encoder_model = Model(inputs=[enc_inp_1, enc_inp_2], outputs=[enc_out, h, c])

In [ ]:
decoder_state_input_h = Input(shape=(32,), name='decoder_input_h')
decoder_state_input_c = Input(shape=(32,), name='decoder_input_c')

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embeddings = dec_emb_layer(decoder_input)

decoder_lstm_1_out, decoder_h1, decoder_c1 = decoder_lstm_1_layer(decoder_embeddings, initial_state=decoder_states_inputs)
decoder_lstm_2_out, decoder_h2, decoder_c2 = decoder_lstm_2_layer(decoder_lstm_1_out)
decoder_lstm_3_out, decoder_h3, decoder_c3 = decoder_lstm_3_layer(decoder_lstm_2_out)
decoder_final_out, decoder_h_final, decoder_c_final = decoder_output_layer(decoder_lstm_3_out)

decoder_states = [decoder_h_final, decoder_c_final]
decoder_attention_out = attention_layer([decoder_final_out, encoder_output])
decoder_combined_output = combined_output_layer([decoder_final_out, decoder_attention_out])
decoder_preds = decoder_dense(decoder_combined_output)

decoder_model = Model(
    [decoder_input] + [decoder_state_input_h, decoder_state_input_c] + [encoder_output],
    [decoder_preds] + decoder_states
)

In [ ]:
def generate_summary(input_text, input_pos, encoder_model, decoder_model, word_to_index_article_extended, word_to_index_pos_extended):
    input_text_seq = txt_to_seq(input_text,word_to_index_article_extended)
    input_text_seq = pad_sequences([input_text_seq],maxlen=text_max,padding='post',value=word_to_index_article_extended['<end>'])
    input_pos_seq = txt_to_seq(input_pos,word_to_index_pos_extended)
    input_pos_seq = pad_sequences([input_pos_seq],maxlen=pad_max,padding='post',value=word_to_index_pos_extended['<end>'])
    encoder_out,h, c = encoder_model.predict([input_text_seq, input_pos_seq])
    states_value=[h,c]
    
    target_seq = np.array([[word_to_index_article_extended['<start>']]])

    stop_condition = False
    decoded_sentence = []
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value + [encoder_out])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = {v: k for k, v in word_to_index_article_extended.items()}.get(sampled_token_index, 'UNK')
    
        if (sampled_word == '<end>' or len(decoded_sentence) >= 50 - 1):
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)
            target_seq = np.array([[sampled_token_index]])
            states_value = [h, c]

    return ' '.join(decoded_sentence)

In [ ]:
t,summary,p = df.sample().iloc[0,:] 

In [ ]:
t

In [ ]:
p

In [ ]:
summary_gen = generate_summary(t, p, encoder_model, decoder_model, word_to_index_article_extended, word_to_index_pos_extended)

In [ ]:
summary

In [ ]:
summary_gen

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.tokenize import word_tokenize

ref_tokens = [word_tokenize(summary)]
cand_tokens = word_tokenize(summary_gen)
bleu_score = sentence_bleu(ref_tokens, cand_tokens)
bleu_score